# [`wordplay` 🎮 💬](https://github.com/saforem2/wordplay): Shakespeare

✍️

[Sam Foreman](https://samforeman.me)
(\[[ALCF](https://alcf.anl.gov/about/people/sam-foreman)\](<https://alcf.anl.gov/about/people/sam-foreman>))  
2025-07-22

We will be using the [Shakespeare
dataset](https://github.com/saforem2/wordplay/blob/main/data/shakespeare/readme.md)
to train a (~ small) 10M param LLM *from scratch*.

<img src="https://github.com/saforem2/wordplay/blob/main/assets/shakespeare.jpeg?raw=true" width="45%" align="center" /><br>

Image generated from
[stabilityai/stable-diffusion](https://huggingface.co/spaces/stabilityai/stable-diffusion)
on [🤗 Spaces](https://huggingface.co/spaces).<br>

<details closed>

<summary>

Prompt Details

</summary>

<ul>

<li>

Prompt:

</li>

<t><q> Shakespeare himself, dressed in full Shakespearean garb, writing
code at a modern workstation with multiple monitors, hacking away
profusely, backlit, high quality for publication </q></t>

<li>

Negative Prompt:

</li>

<t><q> low quality, 3d, photorealistic, ugly </q></t>

</ul>

</details>

## Install / Setup

<b>Warning!</b><br>

**IF YOU ARE EXECUTING ON GOOGLE COLAB**:

You will need to restart your runtime (`Runtime` $\rightarrow\,$
`Restart runtime`)  
*after* executing the following cell:

In [1]:
%%bash

python3 -c 'import wordplay; print(wordplay.__file__)' 2> '/dev/null'

if [[ $? -eq 0 ]]; then
    echo "Has wordplay installed. Nothing to do."
else
    echo "Does not have wordplay installed. Installing..."
    git clone 'https://github.com/saforem2/wordplay'
    python3 wordplay/data/shakespeare_char/prepare.py
    python3 wordplay/data/shakespeare/prepare.py
    python3 -m pip install deepspeed
    python3 -m pip install -e wordplay
fi

/Users/samforeman/projects/saforem2/wordplay/src/wordplay/__init__.py
Has wordplay installed. Nothing to do.

## Post Install

If installed correctly, you should be able to:

``` python
>>> import wordplay
>>> wordplay.__file__
'/path/to/wordplay/src/wordplay/__init__.py'
```

In [2]:
%load_ext autoreload
%autoreload 2
import os
import sys
import ezpz

os.environ['COLORTERM'] = 'truecolor'
if sys.platform == 'darwin':
    # If running on MacOS:
    # os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
    os.environ['TORCH_DEVICE'] = 'cpu'
# -----------------------------------------------

logger = ezpz.get_logger()

import wordplay
logger.info(wordplay.__file__)

## Build Trainer

Explicitly, we:

1.  `setup_torch(...)`
2.  Build `cfg: DictConfig = get_config(...)`
3.  Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4.  Build `trainer = Trainer(config)`

In [3]:
import os
import numpy as np
from ezpz import setup
from hydra.utils import instantiate
from wordplay.configs import get_config, PROJECT_ROOT
from wordplay.trainer import Trainer

HF_DATASETS_CACHE = PROJECT_ROOT.joinpath('.cache', 'huggingface')
HF_DATASETS_CACHE.mkdir(exist_ok=True, parents=True)

os.environ['HF_DATASETS_CACHE'] = HF_DATASETS_CACHE.as_posix()

BACKEND = 'DDP'

rank = setup(
    framework='pytorch',
    backend=BACKEND,
    seed=1234,
)

cfg = get_config(
    [
        'data=shakespeare',
        'model=shakespeare',
        'model.batch_size=1',
        'model.block_size=128',
        'optimizer=shakespeare',
        'train=shakespeare',
        f'train.backend={BACKEND}',
        'train.compile=false',
        'train.dtype=bfloat16',
        'train.max_iters=500',
        'train.log_interval=10',
        'train.eval_interval=50',
    ]
)
config = instantiate(cfg)

### Build `Trainer` object

In [4]:
trainer = Trainer(config)

## Prompt (**prior** to training)

In [5]:
query = "What is an LLM?"
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=16,
    display=False
)
logger.info(f"['prompt']: '{query}'")
logger.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

## Train Model

|  name  |        description         |
|:------:|:--------------------------:|
| `step` |   Current training step    |
| `loss` |         Loss value         |
|  `dt`  | Time per step (in **ms**)  |
| `sps`  |     Samples per second     |
| `mtps` |  (million) Tokens per sec  |
| `mfu`  | Model Flops utilization[1] |

^legend: #tbl-legend

[1] in units of A100 `bfloat16` peak FLOPS

In [6]:
trainer.config.device_type

'cpu'

In [7]:
from rich import print

print(trainer.model)

GPT ( 
 ( transformer ) : ModuleDict ( 
 ( wte ) : Embedding ( 65 , 384 ) 
 ( wpe ) : Embedding ( 128 , 384 ) 
 ( drop ) : Dropout ( p = 0.2 , inplace = False ) 
 ( h ) : ModuleList ( 
 ( 0 - 5 ) : 6 x Block ( 
 ( ln_1 ) : LayerNorm () 
 ( attn ) : CausalSelfAttention ( 
 ( c_attn ) : Linear ( in_features = 384 , out_features = 1152 , bias = False ) 
 ( c_proj ) : Linear ( in_features = 384 , out_features = 384 , bias = False ) 
 ( attn_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ( resid_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ( ln_2 ) : LayerNorm () 
 ( mlp ) : MLP ( 
 ( c_fc ) : Linear ( in_features = 384 , out_features = 1536 , bias = False ) 
 ( act_fn ) : GELU ( approximate = 'none' ) 
 ( c_proj ) : Linear ( in_features = 1536 , out_features = 384 , bias = False ) 
 ( dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ) 
 ) 
 ( ln_f ) : LayerNorm () 
 ) 
 ( lm_head ) : Linear ( in_features = 384 , out_features = 65 , bias = False ) 
 )

## (partial) Training:

We’ll first train for 500 iterations and then evaluate the models
performance on the same prompt:

> What is an LLM?

In [8]:
trainer.train(train_iters=500)

Training Legend 
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ abbr ┃ desc ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ step │ Current training iteration │
│ loss │ Loss value │
│ dt │ Elapsed time per training step │
│ dtf │ Elapsed time per forward step │
│ dtb │ Elapsed time per backward step │
│ sps │ Samples per second │
│ sps_per_gpu │ Samples per second (per GPU) │
│ tps │ Tokens per second │
│ tps_per_gpu │ Tokens per second (per GPU) │
│ mfu │ Model flops utilization │
└─────────────┴────────────────────────────────┘

In [9]:
import time

query = "What is an LLM?"
t0 = time.perf_counter()
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=16,
    display=False
)
logger.info(f'took: {time.perf_counter() - t0:.4f}s')
logger.info(f"['prompt']: '{query}'")
logger.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

## Resume Training…

In [10]:
trainer.train()

## Evaluate Model

In [11]:
import time

query = "What is an LLM?"
t0 = time.perf_counter()
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=2,
    display=False
)
logger.info(f'took: {time.perf_counter() - t0:.4f}s')
logger.info(f"['prompt']: '{query}'")
logger.info("['response']:\n\n" + fr"{outputs['0']['raw']}")